# Day 9: Rope Bridge

## Part 1

This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where not to step.

Consider a rope with a knot at each end; these knots mark the head and the tail of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving Planck lengths, you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical series of motions (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (H) and tail (T) must always be touching (diagonally adjacent and even overlapping both count as touching):
```
....
.TH.
....
```
```
....
.H..
..T.
....
```
```
...
.H. (H covers T)
...
```
If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:
```
.....    .....    .....
.TH.. -> .T.H. -> ..TH.
.....    .....    .....
```
```
...    ...    ...
.T.    .T.    ...
.H. -> ... -> .T.
...    .H.    .H.
...    ...    ...
```
Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:
```
.....    .....    .....
.....    ..H..    ..H..
..H.. -> ..... -> ..T..
.T...    .T...    .....
.....    .....    .....
```
```
.....    .....    .....
.....    .....    .....
..H.. -> ...H. -> ..TH.
.T...    .T...    .....
.....    .....    .....
```
You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:
```
R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
```
This series of motions moves the head right four steps, then up four steps, then left three steps, then down one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

== Initial State ==
```
......
......
......
......
H.....  (H covers T, s)
```
== R 4 ==
```
......
......
......
......
TH....  (T covers s)
```
```
......
......
......
......
sTH...
```
```
......
......
......
......
s.TH..
```
```
......
......
......
......
s..TH.
```
== U 4 ==
```
......
......
......
....H.
s..T..
```
```
......
......
....H.
....T.
s.....
```
```
......
....H.
....T.
......
s.....
```
```
....H.
....T.
......
......
s.....
```
== L 3 ==
```
...H..
....T.
......
......
s.....
```
```
..HT..
......
......
......
s.....
```
```
.HT...
......
......
......
s.....
```
== D 1 ==
```
..T...
.H....
......
......
s.....
```
== R 4 ==
```
..T...
..H...
......
......
s.....
```
```
..T...
...H..
......
......
s.....
```
```
......
...TH.
......
......
s.....
```
```
......
....TH
......
......
s.....
```
== D 1 ==
```
......
....T.
.....H
......
s.....
```
== L 5 ==
```
......
....T.
....H.
......
s.....
```
```
......
....T.
...H..
......
s.....
```
```
......
......
..HT..
......
s.....
```
```
......
......
.HT...
......
s.....
```
```
......
......
HT....
......
s.....
```
== R 2 ==
```
......
......
.H....  (H covers T)
......
s.....
```
```
......
......
.TH...
......
s.....
```
After simulating the rope, you can count up all of the positions the tail visited at least once. In this diagram, s again marks the starting position (which the tail also visited) and # marks other positions the tail visited:
```
..##..
...##.
.####.
....#.
s###..
```
So, there are 13 positions the tail visited at least once.

Simulate your complete hypothetical series of motions. How many positions does the tail of the rope visit at least once?

In [2]:
def parse_instructions(filename):
    """Take an instruction input file and output the instructions as a 2D
    list.

    Args:
        filename (string): Name of the file.

    Returns:
        List of instructions, e.g. [['R', 4], ['U', 4], ...].
    """
    instructions_list = []
    with open(filename) as file:
        lines = file.read().splitlines()
    for line in lines:
        line = line.split(" ")
        instructions_list.append([line[0], int(line[1])])
    return instructions_list

 
def sign(input):
    if input > 0:
        return_value = 1
    elif input < 0:
        return_value = -1
    else:
        return 0
    return return_value


def update_T_based_on_H(H_position, T_position):
     vertical_distance = H_position[0] - T_position[0]
     horizontal_distance = H_position[1] - T_position[1] 
     if abs(vertical_distance) > 1 and horizontal_distance == 0:
         # Vertical move.
         new_T_position = [T_position[0] + sign(vertical_distance), T_position[1]]
     elif abs(horizontal_distance) > 1 and vertical_distance == 0:
         # Horizontal move.
         new_T_position = [T_position[0], T_position[1] + sign(horizontal_distance)]
     elif (abs(horizontal_distance) > 1 and abs(vertical_distance) == 1 or
           abs(horizontal_distance) == 1 and abs(vertical_distance) > 1):
        new_T_position = [T_position[0] + sign(vertical_distance), T_position[1] + sign(horizontal_distance)]
     elif abs(horizontal_distance) > 2 or abs(vertical_distance) > 2:
        raise Exception("H position and T position not close enough for move.")
     else:
        new_T_position = T_position
     return new_T_position
 

def follow_instruction(instruction, H_starting_position, T_starting_position):
    H_position_list = []
    T_position_list = []
    H_previous_position = H_starting_position
    T_previous_position = T_starting_position
    direction = instruction[0]
    num_steps = instruction[1]
    for step in range(num_steps):
        if direction == "U":
            to_append = [H_previous_position[0] + 1, H_previous_position[1]]
            H_position_list.append(to_append)
        elif direction == "D":
            to_append = [H_previous_position[0] - 1, H_previous_position[1]]
            H_position_list.append(to_append)
        elif direction == "R":
            to_append = [H_previous_position[0], H_previous_position[1] + 1]
            H_position_list.append(to_append)
        elif direction == "L":
            to_append = [H_previous_position[0], H_previous_position[1] - 1]
            H_position_list.append(to_append)
        else:
            raise Exception(f"Invalid instruction direction: {direction}.")
        T_position_list.append(update_T_based_on_H(H_position_list[-1], T_previous_position))
        H_previous_position = H_position_list[-1]
        T_previous_position = T_position_list[-1]

    return H_position_list, T_position_list


def main(filename):
    H_position_history = [[0, 0]]
    T_position_history = [[0, 0]]
    instructions = parse_instructions(filename)
    for instruction in instructions:
            H_to_append, T_to_append = follow_instruction(
                instruction,
                H_position_history[-1],
                T_position_history[-1])
            for item in H_to_append:
                H_position_history.append(item)
            for item in T_to_append:
                T_position_history.append(item)
    T_position_history_unique = []
    for entry in T_position_history:
        if entry not in T_position_history_unique:
            T_position_history_unique.append(entry)
    return len(T_position_history_unique)


result = main("input.txt")
print(f"Number of positions: {result}.")

Number of positions: 6197.
